In [3]:
# Import the various modules required to make a simple Snowflake connection from Python
import snowflake.connector
from snowflake.connector.converter_null import SnowflakeNoConverterToPython
import pandas as pd
import os

In [11]:
# Modify this cell to include information about your demo account
ACCOUNT   = os.getenv('SNOW_ACCOUNT')
USER      = os.getenv('SNOW_USER')
PASSWORD  = os.getenv('SNOW_PASSWORD')
WAREHOUSE = os.getenv('SNOW_WAREHOUSE')
ROLE      = os.getenv('SNOW_ROLE')
DATABASE  = os.getenv('SNOW_DATABASE')

con = snowflake.connector.connect(
  user=USER,
  password=PASSWORD,
  account=ACCOUNT,
  warehouse=WAREHOUSE,
  role=ROLE,
  converter_class=SnowflakeNoConverterToPython
)

In [13]:
con.cursor().execute(f"USE DATABASE {DATABASE}")
sql = "create or replace table sample_product_data (id int, parent_id int, category_id int, name varchar, serial_number varchar, key int, \"3rd\" int);"
con.cursor().execute(sql)
sql = """insert into sample_product_data values
    (1, 0, 5, 'Product 1', 'prod-1', 1, 10),
    (2, 1, 5, 'Product 1A', 'prod-1-A', 1, 20),
    (3, 1, 5, 'Product 1B', 'prod-1-B', 1, 30),
    (4, 0, 10, 'Product 2', 'prod-2', 2, 40),
    (5, 4, 10, 'Product 2A', 'prod-2-A', 2, 50),
    (6, 4, 10, 'Product 2B', 'prod-2-B', 2, 60),
    (7, 0, 20, 'Product 3', 'prod-3', 3, 70),
    (8, 7, 20, 'Product 3A', 'prod-3-A', 3, 80),
    (9, 7, 20, 'Product 3B', 'prod-3-B', 3, 90),
    (10, 0, 50, 'Product 4', 'prod-4', 4, 100),
    (11, 10, 50, 'Product 4A', 'prod-4-A', 4, 100),
    (12, 10, 50, 'Product 4B', 'prod-4-B', 4, 100);"""
con.cursor().execute(sql)

In [14]:
# Create a variable called sql and specify a query that it will store
sql = "select * from sample_product_data limit 10000"

In [15]:
# Execute the query using the Python connector
#%%time
res = con.cursor().execute(sql).fetchall()


In [16]:
# Run that same query, but this time use the read_sql method
# in the Pandas data frame object
#%%time
df = pd.read_sql(sql, con)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   ID             12 non-null     int64 
 1   PARENT_ID      12 non-null     int64 
 2   CATEGORY_ID    12 non-null     int64 
 3   NAME           12 non-null     object
 4   SERIAL_NUMBER  12 non-null     object
 5   KEY            12 non-null     int64 
 6   3rd            12 non-null     int64 
dtypes: int64(5), object(2)
memory usage: 800.0+ bytes


In [17]:
df.groupby('PARENT_ID')[['NAME']].count()

,NAME
PARENT_ID,
0,4
1,2
4,2
7,2
10,2


In [18]:
# Check to see if any of the columns have null values
pd.isnull(df).any()

ID               False
PARENT_ID        False
CATEGORY_ID      False
NAME             False
SERIAL_NUMBER    False
KEY              False
3rd              False
dtype: bool

In [19]:
type(res)

list

In [20]:
print (res[0])

(1, 0, 5, 'Product 1', 'prod-1', 1, 10)


In [11]:
unique_cust_key = []
z = []
for x in res:
    z.append((x[0],x[6]))

for x in z:
    if x not in unique_cust_key:
        unique_cust_key.append(x)
    
# initailize a null list 
unique_list = []

# traverse for all elements 
for x in unique_cust_key:
    # check if exists in unique_list or not 
    if x[1] not in unique_list:
        unique_list.append(x[1])
       
def countX(lst, x):
    count = 0
    for y in lst:
        if (y[1] == x):
            count = count + 1
    return count

for a in unique_list:
    print('{} has occured {} times'.format(a, countX(unique_cust_key, a))) 
    


AUTOMOBILE has occured 1974 times
BUILDING has occured 1964 times
MACHINERY has occured 1989 times
HOUSEHOLD has occured 2025 times
FURNITURE has occured 2048 times
